### *PhotoDissociation Region Toolbox Notebooks*
-------------------------------------------------------------

# Example 4: Exploring Models

This example shows use the PDRT Toolbox to look at individual models of ratio or intensity and optionally plot your observations on top. The main class for this is ModelPlot.


In [ ]:
from pdrtpy.modelset import ModelSet
from pdrtpy.plot.modelplot import ModelPlot
from pdrtpy.measurement import Measurement
from astropy.nddata import StdDevUncertainty
import astropy.units as u

In [ ]:
from pdrtpy import version
if version()!='2.0.7-i':
    raise Exception("This notebook is for testing version pdrtpy 2.0.7-i. It will not work yet with other versions.")

### First, load up a set of models. Here we use the Wolfire/Kaufman 2006 models.

In [ ]:
m = ModelSet(name="wk2006",z=1)
# Display all the available ratio and intensity model files.
m.table.show_in_notebook()

### Instantiate a ModelPlot class using the ModelSet you just created.

In [ ]:
mp = ModelPlot(m)

In [ ]:
### Plot a particular model. You identify the model you want using the 'ratio' label in the table above.
mp.plot("CI_609",legend=False,contours=True,label=True)

In [ ]:
### Plot another one.  Add a few keywords for fancification.
mp.plot("H200S2/H200S1",legend=True,cmap='rainbow',norm='log',contours=False)

### Get all the models that match CO(2-1), CO(4-3), and CI 609 micron and plot them. 
The model_type 'both' means get both ratios and intensities.  You could also ask for just 'ratio' or 'intensity'

In [ ]:
z=m.get_models(["CI_609","CO_21","CO_43"],model_type="both")
print("Found models: ",z.keys())
for j in z:
    mp.plot(j,label=True,norm='zscale')

### What if you want to plot observations on the models?
First create some Measurements to represent your observations.

In [ ]:
myunit = "erg s-1 cm-2 sr-1"
mx=Measurement(data=[1.5E-6],uncertainty = StdDevUncertainty(0.5E-6),identifier="CI_609",unit=myunit)
my=Measurement(data=[2],uncertainty = StdDevUncertainty(1),identifier="CII_158/CI_609",unit=myunit)
mz=Measurement(data=[3E-6],uncertainty = StdDevUncertainty(0.5E-6),identifier="CII_158",unit=myunit)
mq=Measurement(data=[2.3],uncertainty=StdDevUncertainty(0.5),identifier='CO_43/CO_21')
a = [mx,my,mz,mq]

### The *overlay* method will plot one or more Measurements in the density-radiation field space.  
The errors are indicated with shading. Note you can pass it both ratios and intensities for the Measurements

In [ ]:
mp.overlay(measurements=a,yaxis_unit="erg s-1 cm-2")

### You can overlay a observation on a model.
Simply pass a Measurement into the *plot* method.

In [ ]:
mp.plot("CII_158/CI_609",image=False,measurements=[my],legend=True,label=True)

### Don't like the default colors?  Use your own!

In [ ]:
mp.plot("CII_158/CI_609",image=False,measurements=[my],legend=True,label=True,meas_color='red',shading=0.3)

### Here is an example plotting only intensities.
The code is smart enough to convert $K~km~s^{-1}$ to intensity units (as long as the Measurement has a RESTFREQ in the FITS header or given during creation) and change the legend label as appropriate.

In [ ]:
ma=Measurement(data=[0.75],uncertainty = StdDevUncertainty(0.3),identifier="CO_65",unit="K km s-1",restfreq=692*u.GHz)
mb=Measurement(data=[1E-6],uncertainty = StdDevUncertainty(0.25E-6),identifier="H200S3",unit=myunit)
mc=Measurement(data=[2E-6],uncertainty = StdDevUncertainty(0.5E-6),identifier="H200S2",unit=myunit)
md=Measurement(data=[1.25E-4],uncertainty = StdDevUncertainty(0.5E-4),identifier="OI_145",unit=myunit)
a = [ma,mb,mc,md]
mp.overlay(measurements=a,shading=0.5)

### Finally, plot them using Draine units on the Y-axis and with dashed lines indicating errors rather than shading.

In [ ]:
mp.overlay(measurements=a,yaxis_unit='Draine',shading=0)